In [1]:
import numpy as np
import random
import json
# new one
class simpleSQL:
    def __init__(self) -> None:
        self.token=[]
        
    # def __init__(self,token = None) -> None:
    #     self.token=token
        
    def add(self,x):
        self.token.append(x)
    
    def toStr(self):
        ans=""
        for i in range(len(self.token)):
            ans+=str(self.token[i].value)
            if i!=len(self.token)-1 \
                and self.token[i].type!="tbname_"\
                    and self.token[i].type!="dot"\
                        and self.token[i].type!="colname_":
                ans+=" "
            # print(str(self.token[i].value)+" ",end="")
        # print()
        return ans+"\n"

class key:
    def __init__(self,value,type) -> None:
        self.value=value
        self.type=type
        # self.name=name
        
    def toStr(self):
        print(f'value : {self.value}')
        print(f'type : {self.type}')


In [2]:
class foreign_constraint:
    def __init__(self,tb1,col1,tb2,col2) -> None:
        self.tb1=tb1
        self.col1=col1
        self.tb2=tb2
        self.col2=col2

In [3]:
class Table:
    def __init__(self,tb_name,col,prim_col,foreign_cons,column_distribution) -> None:
        self.name=tb_name
        self.col=col
        self.col_data_dis={}
        self.prim_col = prim_col
        self.foreign_cons = foreign_cons
        self.column_distribution = column_distribution
    
    def addCharacteristics(self,col_name,data_dis):
        col_name_set=set(self.col[0:len(self.col)])
        if col_name not in col_name_set:
            print("error: add data characteristics failed. Col name not found.")
        else:
            self.col_data_dis[col_name]=data_dis

In [4]:
class DBschema:
    def __init__(self,tbs,foreignConstraint=None) -> None:
        self.tables=tbs
        # self.tbNum=len(tbs)
        self.foreignConstraint=foreignConstraint
    
    def toStr(self):
        ans=""
        for i,j in enumerate(self.tables):
            ans+="table "+str(i+1)
            ans+=" : "+j.name
            ans+="\n"
        return ans
            

In [5]:
# Random sampling in a given probability distribution graph
# ---- return the position(i.e. l+index) corresponding to the sample num.
def rand_num_sampling(l,r,pdg):
    if r-l+1 != len(pdg):
        print("l is "+str(l)+" r is "+str(r)+" and pdg is "+str(pdg))
        print("error : pdg not match [l,r]")
        return l-1
    one_=sum(pdg)
    if abs(one_-1.0)>1e-5:
        print("error : probability sum "+str(one_)+" do not equal 1")
        return l-1
    pdg_=np.array(pdg)
    pdg_=[round(i*1000) for i in pdg_]
    maxv=sum(pdg_)
    num=random.randint(0,maxv-1)
    index=0
    for i in range(len(pdg_)):
        num=num-pdg_[i]
        if num<0:
            index=i
            break
    return l+index

In [6]:
dict={}
for i in range(1,6):
    dict[i]=0
for i in range(1,100000,1):
    # print(randNumGen(1,5,np.full((5),0.2)))
    # dict[randNumGen(1,5,np.full((5),0.2))]+=1
    dict[rand_num_sampling(1,5,[0.1,0.2,0.3,0.4,0.0])]+=1
for i in range(1,6):
    print(dict[i])

9965
19950
29803
40281
0


In [7]:
dict={}
dict[0]=0
dict[1]=0
for i in range(1000):
    dict[rand_num_sampling(0,1,np.array([0.7,0.3]))]+=1
print(dict[0])
print(dict[1])

710
290


In [8]:
# feature1 : size of workload
# feature2 : read/write ratio
# feature3 : average table num
# feature4 : table data distribution
    # tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,condition.avgtb)]
    # tb_choice=get1Table(self.dbs.tables,np.full(len(self.dbs.tables),1.0/len(self.dbs.tables)))
# feature5 : different query comparison constraint ratio
    # cons=randNumGen(0,3,condition.feature4)
# feature6 : query data domain distribution
    # data=randNumGen(0,99,[1.0/100 for i in range(100)])
# feature7 : query logic predicate num
# feature8 : average aggregation operator num
# feature9 : average query colomn num

In [9]:
# data access distribution
class feature:
    def __init__(self,workloadSize,rwRatio,averageTableNum,tableDistribution,queryComparisonOperatorRatio,tableDomainDist,queryLogicPredicateNum,averageAggregationOperatorNum,averageQueryColomnNum) -> None:
        self.workloadSize=workloadSize
        self.rwRatio=rwRatio # feature1
        self.averageTableNum=averageTableNum # feature2
        self.tableDistribution=tableDistribution  # feature3
        self.queryComparisonOperatorRatio = queryComparisonOperatorRatio # feature4
        self.tableDomainDist = tableDomainDist # feature5
        self.queryLogicPredicateNum = queryLogicPredicateNum # feature6
        self.averageAggregationOperatorNum=averageAggregationOperatorNum # feature7
        self.averageQueryColomnNum=averageQueryColomnNum # feature8
        

> ##### update table_name Set column_name = new_value Where column_name = some_value
> ##### insert into table_name [(column1,column2,…)] value (value1,value2,…)
> ##### select {col_name} from table_name where col_name <=> constant

In [10]:
class SQLGen:
    def __init__(self,dbs) -> None:
        self.sql_set=[]
        self.last_sql=simpleSQL()
        self.dbs=dbs

    def generate(self,condition):
        # feature1 : read/write ratio
        rw_choice=rand_num_sampling(0,1,np.array([condition.rwRatio,1-condition.rwRatio]))
        # feature2 : average table num
        tb_num=rand_num_sampling(1,2,condition.averageTableNum)
        # feature3 : table data distribution
        # tb_choice=self.dbs.tables[randNumGen(0,len(self.dbs.tables)-1,[1.0/len(self.dbs.tables) for i in range(len(self.dbs.tables))])]
        # print('aaaa')
        tb_choice=self.dbs.tables[rand_num_sampling(0,len(self.dbs.tables)-1,condition.tableDistribution)]
        # feature4 : different query comparison constraint ratio
        # cons=randNumGen(0,3,[1.0/4 for i in range(4)])
        cons=rand_num_sampling(0,3,condition.queryComparisonOperatorRatio)
        # feature5 : query data domain distribution
        data=rand_num_sampling(0,99,[1.0/100 for i in range(100)])
        if rw_choice==0:
            self.last_sql.add(key(value="select",type="keyword"))
            self.last_sql.add(key(value="*",type="colname"))
            self.last_sql.add(key(value="from",type="keyword"))
            self.last_sql.add(key(value=tb_choice.name,type="tbname"))
            self.last_sql.add(key(value="where",type="keyword"))
            self.last_sql.add(key(value=tb_choice.col[0].value,type="colname"))
            if cons==0:
                self.last_sql.add(key(value=">",type="compare"))
            else:
                if cons==1:
                    self.last_sql.add(key(value="<",type="compare"))
                else:
                    if cons==2:
                        self.last_sql.add(key(value="=",type="compare"))
                    else: 
                        if cons==3:
                            self.last_sql.add(key(value="!=",type="compare"))
                        else: 
                            pass
            self.last_sql.add(key(value=data,type="value"))
            self.last_sql.add(key(value=";",type="end"))
        else:
            self.last_sql.add(key(value="insert",type="keyword"))
            self.last_sql.add(key(value="into",type="keyword"))
            self.last_sql.add(key(value=tb_choice.name,type="tbname"))
            self.last_sql.add(key(value="value",type="keyword"))
            tmp="("
            for i in range(len(tb_choice.col)):
                if i!=0:
                    tmp+=","
                tmp+=str(rand_num_sampling(0,99,[1.0/100 for i in range(100)]));
            tmp+=")"
            self.last_sql.add(key(value=tmp,type="value"))
            self.last_sql.add(key(value=";",type="end"))
        
        self.sql_set.append(self.last_sql)
        return self.last_sql

    def save(self,outputFile):
        with open(outputFile, 'w') as f:
            for it in self.sql_set:
                f.write(it.toStr())
        print(f"workload saved to {outputFile}")

In [11]:
def load_json(filePath,mode):
    jsonFile = open(filePath,mode)
    input = json.loads(jsonFile.read())
    # print(input)
    all_tables = []
    foreign_cons = []
    cons = []
    # load table name & column name
    for table in input['Tables']:
        tb_name = table['Table Name']
        tb_col_distribution = table['Column Distribution']
        tb_cols = []
        for col in table['Table Columns']:
            col_name = col['Column Name']
            col_type = col['Data Type']
            tb_cols.append(key(col_name,col_type))
        
        prim_key = key(table['Primary Key']['Name'],table['Primary Key']['Data Type'])
        for con in table['Foreign Key']:
            foreign_cons.append(foreign_constraint(tb_name,key(con['Foreign Key Name'],con['Foreign Key Type']),
                                                con['Referenced Table'],key(con['Referenced Primary Key'],con['Referenced Primary Key Type'])))
        all_tables.append(Table(tb_name,tb_cols,prim_key,foreign_cons,tb_col_distribution))
    # load constraints
    for con in input['Constraints']:
        cons.append(input['Constraints'][con])
    # print(cons)
    fea = feature(cons[0],cons[1],cons[2],cons[3],cons[4],cons[5],cons[6],cons[7],cons[8])
    #print(cons[0],cons[1],cons[2],cons[3],cons[4])
    dbs=DBschema(tbs=all_tables,foreignConstraint=foreign_cons)
    outputFile=input['Generation File']
    return dbs,fea,outputFile

# cons应该是feature类型
# fea=feature(cons[0],cons[1])
# return DBschema,feature

In [12]:
all_tables,fea,outputFile=load_json('./input.json','r')
table2dist = {}
for i in range(len(all_tables.tables)):
    table2dist[all_tables.tables[i]] = fea.tableDistribution[i]

In [13]:
#tb=Table("lzz",[key("age",int),key("weight",int)])
dbs=all_tables
sg=SQLGen(dbs=dbs)
sg.generate(condition=fea)
print(sg.last_sql.toStr())

select * from tb33 where col5 != 31 ;



In [14]:
def getNTable(dbs,n,pdg):
    # print(n)
    all_tb=[dbs.tables[i] for i in range(len(dbs.tables))]
    # all_tb=[i for i in len(dbs.tables)]
    tb_set=set()
    tb_selected = []
    tb_sub_distribution = []
    if len(all_tb)<n:
        print("fatal error : dbs table num not enough.")
        return -1,set()
    # possibly sample to the same table
    while len(tb_set)<n:
        x=rand_num_sampling(0,len(all_tb)-1,pdg)
        if x!=-1:
            tb_set.add(dbs.tables[x])
            if dbs.tables[x] not in tb_selected:
                tb_selected.append(dbs.tables[x])
                tb_sub_distribution.append(table2dist[dbs.tables[x]])
            else:
                continue
            # print(str(len(tb_set))+"add"+str(fea.tableDistribution[x]))
        else:
            return -1,set()
    # make tb_sub_distribution sum to 1
    sum_of_tsd = 0
    for i in tb_sub_distribution:
        sum_of_tsd = sum_of_tsd + i
    # print(tb_sub_distribution)
    for i in range(len(tb_sub_distribution)):
        tb_sub_distribution[i] = tb_sub_distribution[i]/sum_of_tsd
    # print(tb_sub_distribution)
    return 1,tb_set,tb_sub_distribution

##### test of getNTable

In [15]:
bool,tb_set,tb_dist_set=getNTable(dbs=dbs,n=3,pdg=[0.1,0.3,0.6,0.0])
print(tb_dist_set)

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]


In [16]:
dict={}
for i in range(len(dbs.tables)):
    dict[dbs.tables[i].name]=0
for i in range(10000):
    bool,tb_set,tb_dist_set=getNTable(dbs=dbs,n=2,pdg=[0.1,0.3,0.6,0.0])
    for i in tb_set:
        # print(i.name)
        dict[i.name]+=1
for i in range(len(dbs.tables)):
    print(dict[dbs.tables[i].name])

2909
7838
9253
0


In [17]:
def get1Table(tbs,pdg):
    # print("get1"+str(len(tbs)-1))
    return dbs.tables[rand_num_sampling(0,len(tbs)-1,pdg)]

def get1Column(tb,pdg):
    return tb.col[rand_num_sampling(0,len(tb.col)-1,pdg)]

In [18]:
print(fea.queryLogicPredicateNum)
print(fea.tableDistribution)
print(fea.averageAggregationOperatorNum)

[0.4, 0.4, 0.2]
[0.25, 0.25, 0.25, 0.25]
[0.5, 0.3, 0.2]


##### select [max/avg/sum(col_name) as as_name] from table_name where ...
##### select * from table_name where ... group by col_name
> ##### count/group-by usually involves discrete variables
> ##### sum/max/min/avg usually involves continuious variables

In [19]:
def from_tb_get_continuous_type_column(tbs):
    pass
    
def from_tb_get_discrete_type_column(tbs):
    pass

def from_tb_get_N_columns_for_aggregation(tbs,aggregationNum):
    
    col_set=set()
    while(len(col_set)<aggregationNum):
        tb_choice=tbs[rand_num_sampling(0,len(tbs)-1,np.full(len(tbs),1.0/len(tbs)))]
        col_choice=get1Column(tb_choice,np.full(len(tb_choice.col),1.0/len(tb_choice.col)))
        col_set.add(key(value=col_choice,type=tb_choice))
        # pass
    return list(col_set)

In [20]:
class SQLGen2(SQLGen):
    def generate(self,condition):
        self.last_sql=simpleSQL()
        rw_choice=rand_num_sampling(0,1,np.array([condition.rwRatio,1-condition.rwRatio]))
        if rw_choice==0:
            # read
            tb_num=rand_num_sampling(1,2,condition.averageTableNum)
            
            if tb_num==1:
                tb_choice=get1Table(self.dbs.tables,condition.tableDistribution)
                col_choice=get1Column(tb_choice,tb_choice.column_distribution)
                
                self.last_sql.add(key(value="select",type="keyword"))
                
                on=rand_num_sampling(0,2,condition.averageAggregationOperatorNum)
                aqcn=min(len(tb_choice.col),len(condition.averageQueryColomnNum)-1)
                qcn=rand_num_sampling(0,aqcn,[1.0/(aqcn+1) for i in range(aqcn+1)])
                
                
                if qcn==0:
                    self.last_sql.add(key(value="*",type="colname"))
                else:
                    if col_choice.type=='varchar':
                        aggregation_type=0
                    else:
                        aggregation_type=rand_num_sampling(1,3,[1.0/3 for i in range(3)])
                        
                    if aggregation_type==0:
                        tmp=f'count({col_choice.value}) as count_value_{col_choice.value}'
                    elif aggregation_type==1:
                        tmp=f'avg({col_choice.value}) as average_value_{col_choice.value}'
                    elif aggregation_type==2:
                        tmp=f'min({col_choice.value}) as minimum_value_{col_choice.value}'
                    elif aggregation_type==3:
                        tmp=f'max({col_choice.value}) as maximum_value_{col_choice.value}'
                                
                    self.last_sql.add(key(value=tmp,type="aggregation"))
                    
                self.last_sql.add(key(value="from",type="keyword"))
                self.last_sql.add(key(value=tb_choice.name,type="tbname"))
                self.last_sql.add(key(value="where",type="keyword"))
                
                qc_num=rand_num_sampling(1,3,condition.queryLogicPredicateNum)
                for i in range(qc_num):
                    if i!=0:
                        if rand_num_sampling(0,1,[0.5,0.5])==0:
                            self.last_sql.add(key(value="and",type="predicate"))
                        else:
                            self.last_sql.add(key(value="or",type="predicate"))
                    
                    # tb_choice=get1Table(self.dbs,np.full(len(self.dbs.tables),1.0/len(self.dbs.tables)))
                    # col_choice=get1Column(tb_choice,np.full(len(tb_choice.col),1.0/len(tb_choice.col)))
                    col_choice=get1Column(tb_choice,tb_choice.column_distribution)

                    self.last_sql.add(key(value=tb_choice.name,type="tbname_"))
                    self.last_sql.add(key(value=".",type="dot"))
                    self.last_sql.add(key(value=col_choice.value,type="colname"))
                    
                    cons=rand_num_sampling(0,3,condition.queryComparisonOperatorRatio)
                    if cons==0:
                        self.last_sql.add(key(value=">",type="compare"))
                    elif cons==1:
                        self.last_sql.add(key(value="<",type="compare"))
                    elif cons==2:
                        self.last_sql.add(key(value="=",type="compare"))
                    elif cons==3:
                        self.last_sql.add(key(value="!=",type="compare"))
                    else:
                        pass
                    data=rand_num_sampling(0,99,[1.0/100 for i in range(100)])
                    self.last_sql.add(key(value=data,type="value"))
                
                # self.sql.add(key(value=";",type="end"))
                
            elif tb_num==2:
                # JOIN
                bool,tbs,tbs_sub_distribution=getNTable(dbs=dbs,n=2,pdg=np.full(len(dbs.tables),1.0/len(dbs.tables)))
                tbs_=list(tbs)
                if bool==-1:
                    print("fatal error : dbs sampled failed.")
                else:
                    self.last_sql.add(key(value="select",type="keyword"))
                    
                    on=rand_num_sampling(0,2,condition.averageAggregationOperatorNum)
                    aqcn=len(condition.averageQueryColomnNum)-1
                    qcn=rand_num_sampling(0,aqcn,[1.0/(aqcn+1) for i in range(aqcn+1)])
                    
                    if qcn==0:
                        self.last_sql.add(key(value="*",type="colname"))
                    else:
                        res=from_tb_get_N_columns_for_aggregation(tbs_,qcn)
                        n_col=[res[i].value for i in range(len(res))]
                        n_col_from_tb=[res[i].type for i in range(len(res))]
                        for i,col in enumerate(n_col):
                            if i<on:
                                if col.type=='varchar':
                                    aggregation_type=0
                                else:
                                    aggregation_type=rand_num_sampling(1,3,[1.0/3 for i in range(3)])  
                                if aggregation_type==0:
                                    tmp=f'count({n_col_from_tb[i].name}.{col.value}) as count_value_{col.value}'
                                elif aggregation_type==1:
                                    tmp=f'avg({n_col_from_tb[i].name}.{col.value}) as average_value_{col.value}'
                                elif aggregation_type==2:
                                    tmp=f'min({n_col_from_tb[i].name}.{col.value}) as minimum_value_{col.value}'
                                elif aggregation_type==3:
                                    tmp=f'max({n_col_from_tb[i].name}.{col.value}) as maximum_value_{col.value}'
                                
                                if i!=qcn-1:
                                    self.last_sql.add(key(value=tmp,type="colname_"))
                                else:
                                    self.last_sql.add(key(value=tmp,type="colname"))
                                
                            else:
                                if i!=qcn-1:
                                    self.last_sql.add(key(value=f"{n_col_from_tb[i].name}.{col.value}",type="colname_"))
                                else:
                                    self.last_sql.add(key(value=f"{n_col_from_tb[i].name}.{col.value}",type="colname"))
                            # print(i.value)
                            if i!=qcn-1:
                                self.last_sql.add(key(value=",",type="dot"))

                    self.last_sql.add(key(value="from",type="keyword"))
                    
                    
                    self.last_sql.add(key(value=tbs_[0].name,type="tbname"))
                    self.last_sql.add(key(value="join",type="keyword"))
                    self.last_sql.add(key(value=tbs_[1].name,type="tbname"))
                    self.last_sql.add(key(value="where",type="keyword"))
                    
                    self.last_sql.add(key(value=tbs_[0].name,type="tbname_"))
                    self.last_sql.add(key(value=".",type="dot"))
                    self.last_sql.add(key(value=tbs_[0].col[0].value,type="colname"))
                    
                    self.last_sql.add(key(value="=",type="compare"))
                    
                    self.last_sql.add(key(value=tbs_[1].name,type="tbname_"))
                    self.last_sql.add(key(value=".",type="dot"))
                    self.last_sql.add(key(value=tbs_[1].col[0].value,type="colname"))
                    
                    qc_num=rand_num_sampling(1,3,condition.queryLogicPredicateNum)
                    if qc_num>1:
                        
                        for i in range(qc_num-1):
                            if rand_num_sampling(0,1,[0.5,0.5])==0:
                                self.last_sql.add(key(value="and",type="predicate"))
                            else:
                                self.last_sql.add(key(value="or",type="predicate"))
                            
                            tb_choice=get1Table(tbs_,tbs_sub_distribution)
                            col_choice=get1Column(tb_choice,tb_choice.column_distribution)
                            
                            self.last_sql.add(key(value=tb_choice.name,type="tbname_"))
                            self.last_sql.add(key(value=".",type="dot"))
                            self.last_sql.add(key(value=col_choice.value,type="colname"))
                        
                            cons=rand_num_sampling(0,3,condition.queryComparisonOperatorRatio)
                            if cons==0:
                                self.last_sql.add(key(value=">",type="compare"))
                            elif cons==1:
                                self.last_sql.add(key(value="<",type="compare"))
                            elif cons==2:
                                self.last_sql.add(key(value="=",type="compare"))
                            elif cons==3:
                                self.last_sql.add(key(value="!=",type="compare"))
                            else:
                                pass
                            data=rand_num_sampling(0,99,[1.0/100 for i in range(100)])
                            self.last_sql.add(key(value=data,type="value"))
                        
                            # self.sql.add(key(value=";",type="end"))
                    else:
                        pass
            else:
                pass
            
        elif rw_choice==1:
            tb_choice=self.dbs.tables[rand_num_sampling(0,len(self.dbs.tables)-1,condition.tableDistribution)]
            self.last_sql.add(key(value="insert",type="keyword"))
            self.last_sql.add(key(value="into",type="keyword"))
            self.last_sql.add(key(value=tb_choice.name,type="tbname"))
            self.last_sql.add(key(value="value",type="keyword"))
            tmp="("
            for i in range(len(tb_choice.col)):
                if i!=0:
                    tmp+=","
                tmp+=str(rand_num_sampling(0,99,[1.0/100 for i in range(100)]));
            tmp+=")"
            self.last_sql.add(key(value=tmp,type="value"))
            # self.sql.add(key(value=";",type="end"))
        else:
            pass
        self.sql_set.append(self.last_sql)
        return self.last_sql
    
    def generate_N(self, condition, outputFile):
        self.sql_set=[]
        # ans=list()
        for i in range(condition.workloadSize):
            self.generate(condition)
        
        if condition.workloadSize==len(self.sql_set):
            print("workload generation succeeded.")
            self.save(outputFile)
        else:
            print("fatal error : generation failed. Please check your schema input.")
            return
        
        

In [21]:
# all_tables,cons=loadJson('input.json','r')
# dbs=DBschema(tbs=all_tables,cons=None)
sg=SQLGen2(dbs=dbs)
sg.generate(condition=fea)
print(sg.last_sql.toStr())
sg.generate_N(condition=fea,outputFile=outputFile)

select tb33.col5,tb11.col1 from tb11 join tb33 where tb11.col1 = tb33.col5 and tb22.col3 > 91 and tb22.col3 > 16

